In [ ]:
#рабочая версия от 03.09.2025
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from tkcalendar import DateEntry
import subprocess
import threading
import queue
import logging
import sys
import ctypes

def remove_bot():
    selected_item = tree.selection()
    if selected_item:
        values = tree.item(selected_item)["values"]
        bot_id = values[0]
        answer = messagebox.askyesno("Удаление бота", f"Удалить запись о боте №{bot_id}?")
        if answer:
            tree.delete(selected_item)
            del bots_logs[int(bot_id)]
            reset_selection()


def find_first_available_bot_id():
    current_ids = [int(values[0]) for values in map(lambda x: tree.item(x)["values"], tree.get_children())]
    next_id = 1
    while next_id in current_ids:
        next_id += 1
    return next_id

def launch_bot(bot_id, start_date, counter=1):
    script_name = "ecp_bot.py"
    cmd = [
        sys.executable, "-c",
        rf"""
import sys
sys.path.insert(0, 'C:/Users/User/Anaconda')
from ecp_bot import run_bot
run_bot({bot_id}, '{start_date}', {counter})
        """
    ]
    logger.info(f"Запускаю бот {script_name} #{bot_id} на дату {start_date}.")
    process = subprocess.Popen(cmd, 
                               stdout=subprocess.PIPE, 
                               stderr=subprocess.STDOUT, 
                               universal_newlines=True,
                               encoding='utf-8',  # Explicitly set encoding
                               errors='replace',   # Replace invalid characters instead of crashing
                               )
    tree.insert("", "end", values=(bot_id, start_date, "Работает"))
    ctypes.windll.user32.MessageBeep(-1)
    update_logs_from_process(process, bot_id)

def update_logs_from_process(process, bot_id):
    bots_logs[bot_id] = []
    while True:
        line = process.stdout.readline()
        if not line:
            break
        bots_logs[bot_id].append(line.rstrip())
        log_queue.put((bot_id, line.rstrip()))
    process.wait()
    log_queue.put((bot_id, f"Бот {bot_id} завершил работу."))
    for child in tree.get_children():
        values = tree.item(child)['values']
        if values[0] == bot_id:
            tree.item(child, values=(values[0], values[1], "Завершен"))
            ctypes.windll.user32.MessageBeep(-1)

def select_bot(event):
    global current_selected_bot_id
    item = tree.selection()[0]
    values = tree.item(item)["values"]
    current_selected_bot_id = values[0]
    display_logs(current_selected_bot_id)

def display_logs(bot_id=None):
    logs_text.config(state=tk.NORMAL)
    logs_text.delete("1.0", tk.END)
    if bot_id is None:
        for bid, logs in bots_logs.items():
            for log in logs:
                logs_text.insert(tk.END, f"[Бот {bid}] {log}\n")
    else:
        for log in bots_logs[int(bot_id)]:
            logs_text.insert(tk.END, f"[Бот {bot_id}] {log}\n")
    logs_text.see(tk.END)
    logs_text.config(state=tk.DISABLED)

def poll_log_queue():
    while not log_queue.empty():
        bot_id, msg = log_queue.get_nowait()
        if current_selected_bot_id is None or int(current_selected_bot_id) == bot_id:
            logs_text.config(state=tk.NORMAL)
            logs_text.insert(tk.END, f"[Бот {bot_id}] {msg}\n")
            logs_text.see(tk.END)
            logs_text.config(state=tk.DISABLED)
    root.after(100, poll_log_queue)

def on_launch_click():
    selected_date = date_entry.get_date().strftime("%d.%m.%Y")
    counter_value = counter_var.get()
    bot_id = find_first_available_bot_id()
    threading.Thread(target=launch_bot, args=(bot_id, selected_date, counter_value)).start()
    
def reset_selection():
    global current_selected_bot_id
    current_selected_bot_id = None
    display_logs()
    
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s [%(levelname)s]: %(message)s')
logger = logging.getLogger(__name__)

root = tk.Tk()
root.title("Multi-Bot Manager")

# Верхний фрейм для полей календаря и Counter
top_frame = ttk.Frame(root)
top_frame.grid(row=0, column=0, columnspan=3, sticky="ew", pady=10)
# Виджет календаря
date_entry = DateEntry(top_frame, width=12, background="darkblue", foreground="white", borderwidth=2)
date_entry.grid(row=0, column=0, sticky="w", pady=10, padx=5)
# Поле ввода для Counter
counter_label = ttk.Label(top_frame, text="Counter:")
counter_label.grid(row=0, column=1, sticky="w", padx=5)
counter_var = tk.StringVar(value="1")
counter_entry = ttk.Entry(top_frame, textvariable=counter_var, width=5)
counter_entry.grid(row=0, column=2, sticky="w", padx=5)
# Кнопка запуска бота
btn_launch = ttk.Button(root, text="Запустить бот")
btn_launch.grid(row=1, column=0, columnspan=3, pady=10)
# Средний фрейм для дерева и журнала
middle_frame = ttk.Frame(root)
middle_frame.grid(row=2, column=0, columnspan=3, sticky="nsew")
# Дерево (TreeView) для отображения активных ботов
tree = ttk.Treeview(middle_frame, columns=("#1", "#2", "#3"), show="headings")
tree.heading("#1", text="ID Бота")
tree.heading("#2", text="Стартовая Дата")
tree.heading("#3", text="Статус")
tree.column("#1", anchor=tk.CENTER, stretch=True)
tree.column("#2", anchor=tk.CENTER, stretch=True)
tree.column("#3", anchor=tk.CENTER, stretch=True)
tree.pack(side=tk.LEFT, fill=tk.X, expand=False)
# Прокрутка для дерева
scrollbar_tree = ttk.Scrollbar(middle_frame, orient=tk.VERTICAL, command=tree.yview)
tree.configure(yscrollcommand=scrollbar_tree.set)
scrollbar_tree.pack_forget()
# Поле для вывода логов
logs_text = tk.Text(root, height=10, state=tk.DISABLED)
logs_text.grid(row=3, column=0, columnspan=3, sticky="nsew", pady=10)
# Добавляем прокрутку для логов
scrollbar_logs = ttk.Scrollbar(root, orient=tk.VERTICAL, command=logs_text.yview)
scrollbar_logs.grid(row=3, column=3, sticky="ns")  # Располагаем полосу прокрутки в правой стороне
logs_text.configure(yscrollcommand=scrollbar_logs.set)
# Кнопка сброса выбора
reset_btn = ttk.Button(root, text="Сбросить выбор", command=lambda: reset_selection())
reset_btn.grid(row=4, column=0, columnspan=3, pady=10)
# Кнопка удаления бота
remove_btn = ttk.Button(root, text="Удалить бот", command=remove_bot)
remove_btn.grid(row=5, column=0, columnspan=3, pady=10)

# Настройка растягивания столбцов и рядов
root.rowconfigure(2, weight=0)  # Строка с таблицей статична
root.rowconfigure(3, weight=1)  # Строка с логами растягивается
root.columnconfigure(0, weight=1)

log_queue = queue.Queue()
bots_logs = {}  # Формат: {bot_id: [логи]}
current_selected_bot_id = None
btn_launch.config(command=on_launch_click)
tree.bind("<ButtonRelease-1>", select_bot)
poll_log_queue()
root.mainloop()

2025-11-01 14:00:43,643 [INFO]: Запускаю бот ecp_bot.py #1 на дату 01.11.2025.
Exception in thread Thread-4 (launch_bot):
Traceback (most recent call last):
  File "c:\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_17856\3163211443.py", line 47, in launch_bot
  File "C:\Users\User\AppData\Local\Temp\ipykernel_17856\3163211443.py", line 52, in update_logs_from_process
  File "c:\Python312\Lib\encodings\cp1251.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position 125: character maps to <undefined>


In [1]:
#правка от DeepSeek - пытаемся передать текцщий день бота через логи ---set-current-day---(.+?)---

import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from tkcalendar import DateEntry
import subprocess
import threading
import queue
import logging
import sys
import ctypes
import re

def remove_bot():
    selected_item = tree.selection()
    if selected_item:
        values = tree.item(selected_item)["values"]
        bot_id = values[0]
        answer = messagebox.askyesno("Удаление бота", f"Удалить запись о боте №{bot_id}?")
        if answer:
            tree.delete(selected_item)
            del bots_logs[int(bot_id)]
            del bots_current_day[int(bot_id)]  # Удаляем информацию о текущем дне
            reset_selection()

def find_first_available_bot_id():
    current_ids = [int(values[0]) for values in map(lambda x: tree.item(x)["values"], tree.get_children())]
    next_id = 1
    while next_id in current_ids:
        next_id += 1
    return next_id

def launch_bot(bot_id, start_date, counter=1):
    script_name = "ecp_bot.py"  # Имя файла сценария бота
    cmd = [
        sys.executable, "-c",
        rf"""
import sys
sys.path.insert(0, 'C:/Users/User/Anaconda')
from ecp_bot import run_bot
run_bot({bot_id}, '{start_date}', {counter})
        """
    ]
    logger.info(f"Запускаю бот {script_name} #{bot_id} на дату {start_date}.")
    process = subprocess.Popen(cmd, 
                            stdout=subprocess.PIPE, 
                            stderr=subprocess.STDOUT, 
                            universal_newlines=True,
                            encoding='utf-8',  # Explicitly set encoding
                            errors='replace',   # Replace invalid characters instead of crashing
                            )
    tree.insert("", "end", values=(bot_id, start_date, "Работает", start_date))  # Добавляем столбец с текущим днем
    ctypes.windll.user32.MessageBeep(-1)
    update_logs_from_process(process, bot_id, start_date)

def update_logs_from_process(process, bot_id, start_date):
    bots_logs[bot_id] = []  # Начинаем собирать логи для бота
    bots_current_day[bot_id] = start_date  # Инициализируем текущий день
    
    while True:
        line = process.stdout.readline()
        if not line:
            break
        
        # Проверяем, содержит ли строка маркер смены дня
        day_match = re.search(r'---set-current-day---(.+?)---', line)
        if day_match:
            new_day = day_match.group(1)
            bots_current_day[bot_id] = new_day
            # Обновляем значение в таблице
            for child in tree.get_children():
                values = tree.item(child)['values']
                if values[0] == bot_id:
                    tree.item(child, values=(values[0], values[1], values[2], new_day))
                    break
        
        bots_logs[bot_id].append(line.rstrip())  # Добавляем новую строчку лога
        log_queue.put((bot_id, line.rstrip()))  # Отправляем в очередь
    
    process.wait()
    log_queue.put((bot_id, f"Бот {bot_id} завершил работу."))
    
    for child in tree.get_children():
        values = tree.item(child)['values']
        if values[0] == bot_id:
            tree.item(child, values=(values[0], values[1], "Завершен", values[3]))
            ctypes.windll.user32.MessageBeep(-1)

def select_bot(event):
    global current_selected_bot_id
    item = tree.selection()[0]
    values = tree.item(item)["values"]
    current_selected_bot_id = values[0]
    display_logs(current_selected_bot_id)

def display_logs(bot_id=None):
    logs_text.config(state=tk.NORMAL)
    logs_text.delete("1.0", tk.END)
    if bot_id is None:
        for bid, logs in bots_logs.items():
            for log in logs:
                logs_text.insert(tk.END, f"[Бот {bid}] {log}\n")
    else:
        for log in bots_logs[int(bot_id)]:
            logs_text.insert(tk.END, f"[Бот {bot_id}] {log}\n")
    logs_text.see(tk.END)
    logs_text.config(state=tk.DISABLED)

def poll_log_queue():
    while not log_queue.empty():
        bot_id, msg = log_queue.get_nowait()
        if current_selected_bot_id is None or int(current_selected_bot_id) == bot_id:
            logs_text.config(state=tk.NORMAL)
            logs_text.insert(tk.END, f"[Бот {bot_id}] {msg}\n")
            logs_text.see(tk.END)
            logs_text.config(state=tk.DISABLED)
    root.after(100, poll_log_queue)

def on_launch_click():
    selected_date = date_entry.get_date().strftime("%d.%m.%Y")
    counter_value = counter_var.get()
    bot_id = find_first_available_bot_id()
    threading.Thread(target=launch_bot, args=(bot_id, selected_date, counter_value)).start()

def reset_selection():
    global current_selected_bot_id
    current_selected_bot_id = None
    display_logs()

# Настройка логирования
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s [%(levelname)s]: %(message)s')
logger = logging.getLogger(__name__)

# Создание GUI
root = tk.Tk()
root.title("Multi-Bot Manager")

top_frame = ttk.Frame(root)
top_frame.grid(row=0, column=0, columnspan=4, sticky="ew", pady=10)  # Увеличиваем columnspan до 4

date_entry = DateEntry(top_frame, width=12, background="darkblue", foreground="white", borderwidth=2)
date_entry.grid(row=0, column=0, sticky="w", pady=10, padx=5)

counter_label = ttk.Label(top_frame, text="Counter:")
counter_label.grid(row=0, column=1, sticky="w", padx=5)

counter_var = tk.StringVar(value="1")
counter_entry = ttk.Entry(top_frame, textvariable=counter_var, width=5)
counter_entry.grid(row=0, column=2, sticky="w", padx=5)

btn_launch = ttk.Button(root, text="Запустить бот")
btn_launch.grid(row=1, column=0, columnspan=4, pady=10)  # Увеличиваем columnspan до 4

middle_frame = ttk.Frame(root)
middle_frame.grid(row=2, column=0, columnspan=4, sticky="nsew")  # Увеличиваем columnspan до 4

# Обновляем Treeview для добавления столбца "Текущий день"
tree = ttk.Treeview(middle_frame, columns=("#1", "#2", "#3", "#4"), show="headings")
tree.heading("#1", text="ID Бота")
tree.heading("#2", text="Стартовая Дата")
tree.heading("#3", text="Статус")
tree.heading("#4", text="Текущий день")  # Новый столбец
tree.column("#1", anchor=tk.CENTER, stretch=True)
tree.column("#2", anchor=tk.CENTER, stretch=True)
tree.column("#3", anchor=tk.CENTER, stretch=True)
tree.column("#4", anchor=tk.CENTER, stretch=True)  # Новый столбец
tree.pack(side=tk.LEFT, fill=tk.X, expand=False)

scrollbar_tree = ttk.Scrollbar(middle_frame, orient=tk.VERTICAL, command=tree.yview)
tree.configure(yscrollcommand=scrollbar_tree.set)
scrollbar_tree.pack_forget()

logs_text = tk.Text(root, height=10, state=tk.DISABLED)
logs_text.grid(row=3, column=0, columnspan=4, sticky="nsew", pady=10)  # Увеличиваем columnspan до 4

scrollbar_logs = ttk.Scrollbar(root, orient=tk.VERTICAL, command=logs_text.yview)
scrollbar_logs.grid(row=3, column=4, sticky="ns")  # Сдвигаем полосу прокрутки на одну колонку вправо
logs_text.configure(yscrollcommand=scrollbar_logs.set)

reset_btn = ttk.Button(root, text="Сбросить выбор", command=lambda: reset_selection())
reset_btn.grid(row=4, column=0, columnspan=4, pady=10)  # Увеличиваем columnspan до 4

remove_btn = ttk.Button(root, text="Удалить бот", command=remove_bot)
remove_btn.grid(row=5, column=0, columnspan=4, pady=10)  # Увеличиваем columnspan до 4

root.rowconfigure(2, weight=0)
root.rowconfigure(3, weight=1)
root.columnconfigure(0, weight=1)

# Глобальные переменные
log_queue = queue.Queue()
bots_logs = {}  # Формат: {bot_id: [логи]}
bots_current_day = {}  # Новый словарь для хранения текущего дня каждого бота
current_selected_bot_id = None

btn_launch.config(command=on_launch_click)
tree.bind("<ButtonRelease-1>", select_bot)
poll_log_queue()

root.mainloop()

2025-11-01 15:05:21,320 [INFO]: Запускаю бот ecp_bot.py #1 на дату 01.11.2025.
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Python312\Lib\tkinter\__init__.py", line 1962, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\User\AppData\Local\Temp\ipykernel_15204\1505872383.py", line 92, in select_bot
    item = tree.selection()[0]
           ~~~~~~~~~~~~~~~~^^^
IndexError: tuple index out of range
